In [22]:
!pip install fasttext
!pip install classifier

     |████████████████████████████████| 49 kB 565 kB/s eta 0:00:01
  Created wheel for classifier: filename=classifier-2.0-py3-none-any.whl size=5056 sha256=7bbf91eaa328a801b568d1beaee446bbad001294383f91efb67ac6a0ccc7c151
  Stored in directory: /Users/Natalio/Library/Caches/pip/wheels/87/ea/b6/d1ee201ef021c46f0ae8f1d29bf6e6d039807dfa521a092d5d
Successfully built classifier


In [23]:
import csv
import logging 
import tempfile 
from classifier import SentimentClassifier 
import fasttext 
 
    
    
    
    
    
    

ImportError: cannot import name 'SentimentClassifier' from 'classifier' (/Users/Natalio/miniconda3/envs/ea_env2/lib/python3.7/site-packages/classifier/__init__.py)

In [24]:
logger = logging.getLogger(__name__)
class MLInterface(object):
    CSV_EXPORT_KWARGS = {"sep": ' ', "index": False, "quoting": csv.QUOTE_NONE, "escapechar": ' '}
    CSV_VALID_NAME = '/validation_text.txt'
    CSV_TRAIN_NAME = '/training_text.txt'
    
    def __init__(self, config_model, config_lang, s3_interface):
        self.s3_interface = s3_interface 
        self.tempdir = tempfile.TemporaryDirectory() 
        self.lang = config_lang['lang'] 
        self.model_loss = config_model['loss'] 
        self._lang_detection_model = None 
        self._topic_detection_model = None 
        self._sentiment_detection_model = None 
    
    @property 
    def lang_detection_model(self): 
        if not self._lang_detection_model: 
            self.load_language_detection_model() 
            return self._lang_detection_model 
    
    @property
    def topic_detection_model(self): 
        if not self._topic_detection_model: self.load_topic_detection_model() 
        return self._topic_detection_model 
    
    @property
    def sentiment_detection_model(self): 
        if not self._sentiment_detection_model: 
            self.load_sentiment_detection_model() 
        return self._sentiment_detection_model 
    
    def load_language_detection_model(self): 
        logger.info(f'loading language detection model for language {self.lang}...') 
        self._lang_detection_model = fasttext.load_model( self.s3_interface.get_language_detection_model()) 
        
    def load_topic_detection_model(self): 
        logger.info(f'loading topic detection model...') 
        self._topic_detection_model = fasttext.load_model( self.s3_interface.get_topic_detection_model()) 
    
    def load_sentiment_detection_model(self): 
        logger.info(f'loading sentiment detection model...') 
        if self.lang == 'es': 
            self._sentiment_detection_model = SentimentClassifier() 
        else:
            raise ValueError('choose a language from the available ones: [es]') 
                
    def upload_topic_model(self, model_path): 
        logger.info("uploading trained model to s3...") 
        self.s3_interface.put_topic_model(model_path) 
        
    def train_model(self, train_df, validation_df): 
        model_path = self.tempdir.name + '/' + 'model.ftz' 
        logger.info(f"training topic detection model...") 
        train_df.to_csv(self.tempdir.name + self.CSV_TRAIN_NAME, **self.CSV_EXPORT_KWARGS)
        validation_df.to_csv(self.tempdir.name + self.CSV_VALID_NAME, **self.CSV_EXPORT_KWARGS)
        model = fasttext.train_supervised( input=self.tempdir.name + self.CSV_TRAIN_NAME, autotuneValidationFile=self.tempdir.name + self.CSV_VALID_NAME, loss=self.model_loss ) 
        logger.info("compressing model...") 
        model.quantize() 
        logger.info(f"caching model locally at {model_path}...") 
        model.save_model(model_path) 
        return model_path

In [17]:
model_en = fasttext.train_supervised("/Users/Natalio/Desktop/nlp_associate_ds_test/NLP_Associate_DS_Test/data/training_data/english.txt")

In [4]:
print(model_en.labels)

['__label__Wikipedia', '__label__APR', '__label__PAN11', '__label__Conference_papers']


In [12]:
f = '/Users/Natalio/Desktop/nlp_associate_ds_test/NLP_Associate_DS_Test/data/documents_challenge/Wikipedia/en/159755156-en.txt'
#doc = open(f, encoding='utf-8').read()
label = model_en.predict(f)

In [13]:
label

(('__label__Conference_papers',), array([1.00001001]))